![1](images/FEMTO_FOREST_500_1.png)
    
![2](images/FEMTO_FOREST_500_2.png)

![3](images/FEMTO_FOREST_500_3.png)
    
<!-- ![4](images/FEMTO_FOREST_500_4.png) -->

In [1]:
import itertools
import json
import pennylane as qml
import pennylane.numpy as np
# You can use auxiliary functions if you are more comfortable with them
# Put your code here #

def circuit(oracle):
    """
    Circuit whose output will determine the team that will carry out the project.

    Args:
        - oracle (callable): the oracle to use in the circuit. To use it, you can write ``oracle(wires = <wires>))``

    You do not have to return anything, just write the gates you need.
    """

    # Put your code here #
    # qml.PauliX(8)
    # for i in range(8):
    #     for j in range(i+1,8):
    #         qml.PauliX(i)
    #         qml.PauliX(j)
    #         oracle(wires = [0,1,2,3,4,5,6,7,8])
    #         qml.CNOT([8,i])
    #         qml.CNOT([8,j])
    # qml.PauliX(8)
    for wire in range(8):
        qml.Hadamard(wires=wire)
    U = np.zeros((2**9,2**9))
    for i in range(len(U)):
      for j in range(len(U)):
        if i%2 ==0 :
          if i == j and i%2 == 0:
            U[i][j] = -0.5
          if j%2 == 0 and i !=j:
            U[i][j] = 0.5
        if i == j and i%2 != 0:
          U[i][j] = -1.0
    for _ in range(int(np.floor((np.pi/4)*np.sqrt(2**8)))):
        oracle(wires = [0,1,2,3,4,5,6,7,8])
        qml.PauliZ([8])
        qml.QubitUnitary(U,wires=[0,1,2,3,4,5,6,7,8])
    

# These functions are responsible for testing the solution.

def run(case: str) -> str:
    workers = json.loads(case)

    def oracle_maker():
        """
        This function will create the Project oracle of the statement from the list of non-lazy workers.

        Returns:
            callable: the oracle function
        """

        def oracle(wires):

            class op(qml.operation.Operation):
                num_wires = 9
                grad_method = None

                def __init__(self, wires, id=None):
                    super().__init__(wires=wires, id=id)

                @property
                def num_params(self):
                    return 0

                @staticmethod
                def compute_decomposition(wires):
                    n_workers = 8
                    matrix = np.eye(2 ** n_workers)

                    for x in range(2 ** n_workers):
                        bit_strings = np.array([int(i) for i in f"{x:0{n_workers}b}"])
                        if sum(bit_strings[workers]) > 1:
                            matrix[x, x] = -1

                    ops = []
                    ops.append(qml.Hadamard(wires=wires[-1]))
                    ops.append(qml.ctrl(qml.QubitUnitary(matrix, wires=wires[:-1]), control=wires[-1]))
                    ops.append(qml.Hadamard(wires=wires[-1]))

                    return ops

            return op(wires=wires)

        return oracle

    dev = qml.device("default.qubit", wires=9)
    oracle = oracle_maker()
    @qml.qnode(dev)
    def circuit_solution(oracle):
        circuit(oracle)
        return qml.probs(wires = range(8))

    return json.dumps([float(i) for i in circuit_solution(oracle)] + workers)


def check(have: str, want: str) -> None:
    have = json.loads(have)
    probs = have[:2**8]
    workers = have[2**8:]
    sol = 0
    n_workers = 8
    for x in range(2 ** n_workers):
        bit_strings = np.array([int(i) for i in f"{x:0{n_workers}b}"])
        if sum(bit_strings[workers]) == 2:
            num_dec = int(''.join(map(str, bit_strings)), 2)
            sol += probs[num_dec]

    assert sol >= 0.95, "The probability success is less than 0.95"
